In [ ]:
!pip install  cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.8 MB/s eta 0:00:00


In [ ]:
import cohere

In [ ]:
co = cohere.ClientV2("egFS8BVGz1DDbYewbj0Tjzh7fUByUIEzflL2GCsM")

# Step 1: Quickstart

In [ ]:

#add a message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."

#Generate a response
response = co.chat(model="command-r-plus-08-2024", messages= [{"role": "user","content" : message}])

print(response.message.content[0].text)

Here's a draft of an introduction message for your new colleagues at Co1t:

"Hello everyone!

My name is [Your Name], and I am thrilled to introduce myself as the newest member of the Co1t team! Starting my journey here as [Your Role/Position] is an exciting opportunity, and I am eager to contribute to the company's success.

A little about myself: I have [mention your relevant experience or background related to the startup's industry]. I am passionate about [list some of your interests or skills related to the job] and believe that my expertise can help drive innovation within the startup environment.

Joining Co1t feels like a perfect fit, as I am impressed by the company's vision and the dynamic team culture. I look forward to collaborating with all of you, learning from your unique perspectives, and sharing ideas. Let's connect and discuss ways to make our startup journey a remarkable one!

Feel free to reach out if you'd like to grab a coffee (or virtual tea!) and chat further. I

# Step 2: Defining a Custom System Message

A conversation starts with a system message, or a preamble, to help steer a chatbot’s response toward certain characteristics. For example, if we want the chatbot to adopt a formal style, the preamble can be used to encourage the generation of more business-like and professional responses.

The custom system message and the current user message form the prompt to the chatbot

In the earlier example, we didn’t have to define a custom system message because a default one was used. We can, however, define our own by adding a message with the system role in the messages array.

In [ ]:
# Create a custom system message
system_message="""

## Task and Context
You are an assistant who assist new employees of Co1t with their first week.

## Style Guide
Try to speak in rhymes as much as possible. Be professional."""

messages = [{"role": "system", "content" : system_message},
            {"role": "user", "content": message}]

In [ ]:
response = co.chat(model = "command-r-plus-08-2024", messages=messages )

print(response.message.content[0].text)

Welcome aboard, a new journey awaits!
At Co1t, we're thrilled to have you join our ranks.

Here's an introduction, a little verse,
To break the ice and quench your thirst.

Hello, my new friends, a pleasure to meet,
[Your Name] here, ready to greet.
I'm excited to embark on this adventure so sweet.
With skills [Your expertise or role], I'll contribute to our feat.

Let's connect and collaborate, a team so bright,
Together, we'll conquer and shine so light.
Feel free to reach out, I'm here to lend an ear,
Let's make Co1t thrive, and have no fear.

A new chapter begins, with a friendly cheer!


# Step 3: Streaming the Chatbot Response

Our examples so far generate responses in a non-streamed manner. This means that the endpoint would return a response object only after the model has generated the text in full. The longer the text is, the longer it takes to get back the response. If you are building an application, this directly impacts the user’s perception of the application’s latency.

The Chat endpoint solves this problem by supporting streamed responses. In a streamed response, the endpoint would return a response object for each token as it is being generated. This means you can display the text incrementally without having to wait for the full completion.

To activate it, use co.chat_stream() instead of co.chat().

In [ ]:
response = co.chat_stream(model = "command-r-plus-08-2024", messages= [{"role": "user", "content": message}])

for chunk in response:
  if chunk.type == "content-delta":
    print(chunk.delta.message.content.text, end = "")

Exception ignored in: <generator object V2Client.chat_stream at 0x7bf83a0c0820>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
RuntimeError: generator ignored GeneratorExit


Here's a draft of an introduction message for your new colleagues at Co1t:

"Hello everyone!

My name is [Your Name], and I am thrilled to introduce myself as the newest member of Team Co1t! I am excited to be joining this innovative startup on its journey to success. As I begin this new chapter, I wanted to take a moment to share a bit about myself.

A little background about me: I have always been passionate about [insert relevant field/industry] and have [X years] of experience working in [specific roles or projects related to the industry]. I am particularly interested in [mention any specialized skills or areas of expertise] and am eager to contribute my knowledge to the team. My enthusiasm for [talk about a recent trend or technology in the industry] brought me to Co1t, as I believe the company's vision aligns perfectly with my own aspirations.

I am looking forward to collaborating with all of you and learning from your diverse skill sets. One of my strengths is [mention a soft 

# Step 4: Building the Chat History

At the core of a conversation is a multi-turn dialog between the user and the chatbot. This requires the chatbot to have a “memory” of all the previous turns to maintain the state of the conversation.

The messages list is used to store the conversation history, including both user messages and chatbot responses. It is structured as an array of objects, with each object representing a message in the conversation.

By appending the previous response and new user message to messages, the model can access the chat history and generate contextually-aware responses. This allows the chatbot to refer to previous messages and maintain a consistent conversation flow.

To illustrate this, let's look at an example. We start with the first conversation turn.

Let's also add a custom system message for generating concise response, just to keep the outputs brief for this tutorial.

In [ ]:
#add a user message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."


# Create a custom system message
system_message="""## Task and Context
Generate concise responses, with maximum one-sentence."""

messages = [  {"role": "user", "content": message},
            {"role": "system","content": system_message} ]


response = co.chat(model = "command-r-plus-08-2024", messages=messages)

print(response.message.content[0].text)

"Hi everyone! I'm thrilled to join Co1t as a new team member, and I look forward to contributing to our shared vision and success."


Now, we want the model to refine the earlier response. This requires the next generation to have access to the state, or memory, of the conversation.

To do this, we append the message object from the previous response to the messages list.

Then we append the next user message to the messages list.

Looking at the response, we see that the model is able to get the context from the chat history. The model is able to capture that "it" in the user message refers to the introduction message it had generated earlier.

In [ ]:
# Option 1: Reset the messages list

messages = [  {"role": "user", "content": "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."},
            {"role": "system","content": "## Task and Context\nGenerate concise responses, with maximum one-sentence."} ]

# Generate the response with the current chat history as the context
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

print("\n")
# Append the previous response - ensuring the correct role name
messages.append({"role": "assistant", "content": response.message.content})

#add the user
message = "make it more upbeat and conversational"

#append the user message
messages.append({"role": "user", "content" : message})

# Generate the response with the current chat history as the context
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hi everyone! I'm thrilled to join Co1t as a new team member, and I look forward to contributing my skills and collaborating with this talented group."


"Hey, Co1t crew! Super excited to be a part of this awesome startup journey. Can't wait to get to know you all and dive into some cool projects together!"


You can continue doing this for any number of turns by repeating the same steps of appending the chatbot and user messages.

In [ ]:
# Append the previous response
messages.append({'role': 'assistant', 'content': response.message.content})

# Add the user message
message = "Thanks. Could you create another one for my DM to my manager."

# Append the user message
messages.append({'role': 'user', 'content': message})

# Generate the response with the current chat history as the context
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hey [Manager's Name], just wanted to express my excitement about starting at Co1t. Looking forward to learning and growing under your guidance!"


# To look at the current chat history, you can print the messages list.

In [ ]:
# Append the previous response
messages.append({'role': 'assistant', 'content': response.message.content})
# View the chat history
for message in messages:
    print(message,"\n")

{'role': 'user', 'content': "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."} 

{'role': 'system', 'content': '## Task and Context\nGenerate concise responses, with maximum one-sentence.'} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentItem(type='text', text='"Hi everyone! I\'m thrilled to join Co1t as a new team member, and I look forward to contributing my skills and collaborating with this talented group."')]} 

{'role': 'user', 'content': 'make it more upbeat and conversational'} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentItem(type='text', text='"Hey, Co1t crew! Super excited to be a part of this awesome startup journey. Can\'t wait to get to know you all and dive into some cool projects together!"')]} 

{'role': 'user', 'content': 'Thanks. Could you create another one for my DM to my manager.'} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentIt